<a href="https://colab.research.google.com/github/CodeCraftIA/netflix_scrape/blob/main/netflix_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import requests
from bs4 import BeautifulSoup

url ="https://www.netflix.com/us/title/81466211"

headers = {"User-Agent": "Content-Scraping"}
response = requests.get(url, headers=headers)

if not response.ok:
    print('Status Code:', response.status_code)
    raise Exception('Failed to fetch: ', url)

soup = BeautifulSoup(response.text)

Status Code: 404


Exception: ('Failed to fetch: ', 'https://www.netflix.com/watch/81555066?tctx=19%2C0%2C%2C%2C%2C%2C%2C%2C%2CVideo%3A81555066%2C')

In [ ]:
infos = soup.find('div', class_="title-info-metadata-wrapper")
if infos:
  year = infos.find('span', class_="title-info-metadata-item item-year")
  maturity=infos.find('span',class_="title-info-metadata-item item-maturity")
  duration = infos.find('span', class_="title-info-metadata-item item-runtime")
  genre = infos.find('a', class_="title-info-metadata-item item-genre")
  print("year: ", year.text)
  print("maturity: ", maturity.text)
  print("duration: ", duration.text)
  print("genre: ", genre.text)
description = soup.find('div',class_="title-info-synopsis")
print("description: ", description.text)



In [ ]:
# Find all genre elements
genre_elements = soup.find_all("a", class_="more-details-item item-genres")

# Extract the text from each genre element and combine them into a single string
genres = ', '.join([genre.get_text() for genre in genre_elements])

# Print the result
print(genres)

Documentary Films, Biographical Documentaries


In [ ]:
# Find all genre elements
cast_elements = soup.find_all("span", class_="more-details-item item-cast")

# Extract the text from each genre element and combine them into a single string
cast = ', '.join([c.get_text() for c in cast_elements])

# Print the result
print(cast)

Robert Downey Sr., Robert Downey Jr.


In [ ]:
mood_elements = soup.find_all("span", class_="more-details-item item-mood-tag")
# Extract the text from each genre element and combine them into a single string
mood = ''.join([m.get_text() for m in mood_elements])

# Print the result
print(mood)



Intimate, Emotional


In [ ]:
trailer12 = soup.find('span', class_="additional-video-title")
trailer12.text

'Trailer: "Sr."'

Start the proccess

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm  # Use notebook tqdm for Colab
from urllib.parse import urlparse, parse_qs

def scrape_data(url):
    headers = {"User-Agent": "Content-Scraping"}
    response = requests.get(url, headers=headers)

    if not response.ok:
        print('Status Code:', response.status_code)
        return None  # Return None to indicate failure

    soup = BeautifulSoup(response.text, 'html.parser')

    year =""
    maturity =""
    duration =""
    infos = soup.find('div', class_="title-info-metadata-wrapper")
    if infos:
      year = infos.find('span', class_="title-info-metadata-item item-year")
      if year:
        year = year.text
      maturity=infos.find('span',class_="title-info-metadata-item item-maturity")
      if maturity:
        maturity = maturity.text
      duration = infos.find('span', class_="title-info-metadata-item item-runtime")
      if duration:
        duration = duration.text
    description = soup.find('div',class_="title-info-synopsis")
    if description:
        description = description.text
    else:
      description=""

    # Find all genre elements
    genre_elements = soup.find_all("a", class_="more-details-item item-genres")
    if genre_elements:
      # Extract the text from each genre element and combine them into a single string
      genres = ', '.join([genre.get_text() for genre in genre_elements])
    else:
      genres=""

    # Find all genre elements
    cast_elements = soup.find_all("span", class_="more-details-item item-cast")
    if cast_elements:
      # Extract the text from each genre element and combine them into a single string
      cast = ', '.join([c.get_text() for c in cast_elements])
    else:
      cast=""

    mood_elements = soup.find_all("span", class_="more-details-item item-mood-tag")
    if mood_elements:
      # Extract the text from each genre element and combine them into a single string
      mood = ''.join([m.get_text() for m in mood_elements])
    else:
      mood=""

    trailer12 = soup.find('span', class_="additional-video-title")
    if trailer12:
      trailer12=trailer12.text
    else:
      trailer12=""

    parsed_url = urlparse(url)
    netflix_id = parsed_url.path.split('/')[-1]  # Get the last part of the path, which should be the ID
    # Construct the trailer URL
    trailer_url = f"https://www.netflix.com/watch/{netflix_id}"
    trailer_url
    # Return a simplified dictionary for this example
    return {
        'Year': year,
        'Length': duration,
        'Description': description,
        'Genres (text)': genres,
        'This film is (text)': mood,
        'Cast': cast,
        'Maturity rating 2/2': maturity,
        'Trailers & More': trailer12,
        'Trailers & More link': trailer_url,
    }


In [ ]:
import pandas as pd
import time
import random

def update_excel_with_scraped_data(excel_path, error_log_path):
    # Load the Excel file
    df = pd.read_excel(excel_path)

    # Create or clear the error log
    open(error_log_path, 'w').close()

    # Iterate over the DataFrame
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        url = row['Title URL']
        try:
            scraped_data = scrape_data(url)
            if scraped_data:
                # Update the DataFrame with the scraped data
                for key in scraped_data:
                    df.at[index, key] = scraped_data[key]
            else:
                # Log the row number (index + 1 for human-readable format)
                with open(error_log_path, 'a') as error_file:
                    error_file.write(f"Row: {index + 1}\n")
        except Exception as e:
            print(f"Error scraping data for URL: {url}\nError: {e}")
            with open(error_log_path, 'a') as error_file:
                error_file.write(f"Row: {index + 1}\n")
        # Random sleep between 1 to 5 seconds
        time.sleep(random.randint(1, 5))
    # Save the updated DataFrame back to the Excel file
    df.to_excel(excel_path, index=False)


In [ ]:
excel_path = '/content/drive/MyDrive/Crawl_2024_03_25_23_44.xlsx'  # Update with your path
error_log_path = '/content/drive/MyDrive/errors.txt'  # Update with your path
update_excel_with_scraped_data(excel_path, error_log_path)


  0%|          | 2/1452 [00:07<1:27:56,  3.64s/it]

Status Code: 404


  0%|          | 4/1452 [00:13<1:11:34,  2.97s/it]

Status Code: 404


  0%|          | 5/1452 [00:15<1:05:14,  2.71s/it]

...

 99%|█████████▉| 1440/1452 [1:23:42<00:34,  2.86s/it]

Status Code: 404


 99%|█████████▉| 1443/1452 [1:23:55<00:35,  3.99s/it]

Status Code: 404


100%|██████████| 1452/1452 [1:24:30<00:00,  3.49s/it]


Continue to the errors

In [ ]:
def update_excel_with_scraped_data(excel_path):
    # Load the Excel file
    df = pd.read_excel(excel_path)

    # Iterate over the DataFrame
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        year_value = row['Year']
        # Check if the "Year" value is empty (NaN) or None
        if not (pd.isnull(year_value) or year_value == ""):
          continue


        url = row['Title URL']
        parsed_url = urlparse(url)
        netflix_id = parsed_url.path.split('/')[-1]  # Get the last part of the path, which should be the ID
        url = "https://www.netflix.com/gr-en/title/"+netflix_id
        try:
            scraped_data = scrape_data(url)
            if scraped_data:
                for key, value in scraped_data.items():
                    df.at[index, key] = value
        except Exception as e:
            print(f"Error scraping data for URL: {url}\nError: {e}")
            # If there's an error, the index remains in the set, so no need to re-add it

        time.sleep(4)

    # Save the updated DataFrame back to the Excel file
    df.to_excel(excel_path, index=False)

excel_path = '/content/drive/MyDrive/Crawl_2024_03_25_23_44.xlsx'  # Update with your path
update_excel_with_scraped_data(excel_path)


 12%|█▏        | 169/1452 [05:42<57:27,  2.69s/it]

Status Code: 404


 12%|█▏        | 173/1452 [05:46<39:57,  1.87s/it]

Status Code: 404


 24%|██▎       | 344/1452 [11:16<43:50,  2.37s/it]

...

 98%|█████████▊| 1429/1452 [45:24<00:44,  1.94s/it]

Status Code: 404


 99%|█████████▉| 1436/1452 [45:37<00:32,  2.00s/it]

Status Code: 404


100%|██████████| 1452/1452 [45:55<00:00,  1.90s/it]


update

In [ ]:
import pandas as pd

def update_excel(excel_path):
    # Load the Excel file
    df = pd.read_excel(excel_path)

    # Update the column 'Maturity rating 2/2'
    df['Maturity rating 2/2'] = df['Maturity rating 2/2'].str.replace('Maturity Rating:', '')

    # Save the updated DataFrame back to the Excel file
    df.to_excel(excel_path, index=False)

# Call the function to update the Excel file
update_excel("/content/drive/MyDrive/Crawl_2024_03_25_23_44.xlsx")
